<a href="https://colab.research.google.com/github/dcatrib/tesePython/blob/main/Scrap_e_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instala os pacotes

In [1]:
!pip install PyMuPDF requests selenium chromedriver-autoinstaller pandas nltk gensim numpy xlrd


!apt-get update
!apt-get install -y wget
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 23.9 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:1

Importa os pacotes

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import chromedriver_autoinstaller
import requests
import fitz
import os
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from google.colab import files
import numpy as np
from gensim.models import KeyedVectors, Word2Vec
import gensim
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AutoReg


Configura o webdriver

In [3]:
# Configurando o WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Instala automaticamente a versão correta do ChromeDriver
chromedriver_autoinstaller.install()



'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/131/chromedriver'

Entra no url, espera os seletores e armazena os links

In [4]:
# URL para scrape
url = "https://www.bcb.gov.br/en/publications/copomminutes/cronologicos"

# Criar uma nova instância do Chrome driver
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

# Esperar o seletor específico estar presente
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'body > app-root > app-root > main > dynamic-comp > div > div > bcb-publicacao > div > div > bcb-ultimaspublicacoes > div'))
)

# Definir o seletor
selector = 'body > app-root > app-root > main > dynamic-comp > div > div > bcb-publicacao > div > div > bcb-ultimaspublicacoes > div'

# Lista para armazenar hrefs, textos e links de download
hrefs_and_texts = []


Entra no link e baixa o conteudo do pdf

In [5]:
# Função para recuperar links e botões de download
def retrieve_links():
    try:
        elements = driver.find_elements(By.CSS_SELECTOR, selector)
        link_count = 0  # Inicializa um contador para os links
        for element in elements:
            links = element.find_elements(By.TAG_NAME, 'a')
            for link in links:
                if link_count >= 183:  # Limita a 50 links
                    return  # Sai da função se 50 links foram processados
                href = link.get_attribute('href')
                text = link.text
                if href and text and text[0].isdigit():
                    driver.get(href)
                    # Esperar o botão de download
                    try:
                        download_button = WebDriverWait(driver, 65).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, '#publicacao > div.col-lg-9.d-flex.flex-column > div > div > div > div.d-flex.flex-column-reverse.flex-sm-row.justify-content-sm-between.align-items-sm-center > div.d-flex.align-items-center.mb-3.mb-sm-0.d-print-none > download > div > div > a'))
                        )
                        download_href = download_button.get_attribute('href')

                        # Fazer download do PDF
                        pdf_response = requests.get(download_href)
                        pdf_filename = f"{text}.pdf"
                        with open(pdf_filename, 'wb') as pdf_file:
                            pdf_file.write(pdf_response.content)

                        # Extrair texto do PDF usando PyMuPDF
                        pdf_text = ''
                        with fitz.open(pdf_filename) as pdf_file:
                            for page in pdf_file:
                                page_text = page.get_text()
                                pdf_text += page_text + '\n'

                        # Limpeza do texto
                        pdf_text = pdf_text.replace('\n', ' ')  # Remove quebras de linha
                        pdf_text = re.sub(r'\s+', ' ', pdf_text)  # Substitui múltiplos espaços por um único
                        pdf_text = pdf_text.strip()  # Remove espaços no início e no fim

                        # Print da primeira linha extraída
                        first_line = pdf_text.split('.')[0]  # Considera a primeira frase
                        #print(f'Primeira linha extraída: {first_line.strip()}')
                        print(link_count)
                    except (NoSuchElementException, TimeoutException):
                        download_href = None
                        pdf_text = None

                    hrefs_and_texts.append((href, text, download_href, pdf_text))
                    link_count += 1  # Incrementa o contador
                    driver.back()
                    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
    except StaleElementReferenceException:
        retrieve_links()

# Iniciar a recuperação de links
retrieve_links()

# Fechar o driver
driver.quit()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182


Minhas stopwords e expressoes

In [6]:
# Minhas stopwords
my_stopwords = {
    'January', 'February', 'March', 'April', 'May', 'June', 'July',
    'August', 'September', 'October', 'November', 'December','january', 'february', 'march', 'april', 'may', 'june', 'july',
    'august', 'september', 'october', 'november', 'december', 'pp',
    'minutes', 'bcb', 'th', 'copom', 'bcbgovbr', 'brasilia', 'pm','roberto', "twelve", 'month', 'monthonmonth', 'monthly',
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "us", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "would", "could", "should", "ought", "not", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "one", "every", "least", "less", "many", "now", "ever", "never", "say", "says", "said", "also", "get", "go", "goes", "just", "made", "make", "put", "see", "seen", "whether", "like", "well", "back", "even", "still", "way", "take", "since", "another", "however", "two", "three", "four", "five", "first", "second", "new", "old", "high", "long"
}

# Array de expressões a serem removidas
expressions_to_remove = [
    'bcb.gov.br Minutes of the Meeting of the Monetary Policy Committee — Copom',
]

fazer o preprocessamento e criar o df

In [7]:
# Baixar as stopwords se ainda não tiver feito
nltk.download('stopwords')

# Criar um DataFrame com os links e textos extraídos
df = pd.DataFrame(hrefs_and_texts, columns=['Link', 'Texto', 'Link de Download', 'Texto do PDF'])

# Carregar stopwords em inglês
stop_words = set(stopwords.words('english'))
stop_words.update(my_stopwords)

# Função para preprocessar o texto
def preprocess_text(text):
    if text:
        # Remover números, pontuação e caracteres especiais
        text = re.sub(r'[0-9]+', '', text)  # Remove números
        # Remover pontuação, exceto parênteses
        text = re.sub(r'[^\w\s()]+', '', text)  # Mantém letras, números, espaços e parênteses
        text = re.sub(r'\s+', ' ', text)  # Substituir múltiplos espaços por um único espaço
        text = text.strip().lower()  # Retorna texto em minúsculas

        # Remover expressões específicas
        for expr in expressions_to_remove:
            text = text.replace(expr, '')

        # Remover stopwords
        text = ' '.join([word for word in text.split() if word not in stop_words])
        return text
    return ''

# Criar a nova coluna 'Texto Processado'
df['Texto Processado'] = df['Texto do PDF'].apply(preprocess_text)

# Imprimir o DataFrame
print(df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


                                                  Link  \
0    https://www.bcb.gov.br/en/publications/copommi...   
1    https://www.bcb.gov.br/en/publications/copommi...   
2    https://www.bcb.gov.br/en/publications/copommi...   
3    https://www.bcb.gov.br/en/publications/copommi...   
4    https://www.bcb.gov.br/en/publications/copommi...   
..                                                 ...   
178  https://www.bcb.gov.br/en/publications/copommi...   
179  https://www.bcb.gov.br/en/publications/copommi...   
180  https://www.bcb.gov.br/en/publications/copommi...   
181  https://www.bcb.gov.br/en/publications/copommi...   
182  https://www.bcb.gov.br/en/publications/copommi...   

                                     Texto  \
0       266th Meeting - November 5-6, 2024   
1    265th Meeting - September 17-18, 2024   
2         264th Meeting - July 30-31, 2024   
3         263rd Meeting - June 18-19, 2024   
4            262nd Meeting - May 7-8, 2024   
..                         

Extrai do texto a data da reuniao, formata a data e coloca como uma variavel no df

In [8]:
# Função para extrair ano e mês
def extract_year_month(text):
    # Caso 1: Formato com dia e mês
    match_full = re.search(r'(\w+)\s+\d{1,2}[- ]\d{1,2},\s*(\d{4})', text)
    # Caso 2: Formato apenas com mês e ano
    match_month_year = re.search(r'(\w+)\s+(\d{4})', text)
    # Caso 3: Formato com intervalo de datas
    match_interval = re.search(r'(\w+)\s+\d{1,2}\s*-\s*(\w+)\s+\d{1,2},\s*(\d{4})', text)

    if match_full:
        month_str = match_full.group(1)
        year = match_full.group(2)
    elif match_month_year:
        month_str = match_month_year.group(1)
        year = match_month_year.group(2)
    elif match_interval:
        month_str = match_interval.group(2)  # Captura o mês da última data
        year = match_interval.group(3)
    else:
        return None

    # Converter mês para número
    month_num = pd.to_datetime(month_str, format='%B').month
    return f"{year}-{month_num:02d}"  # Formatar como YYYY-MM

# Criar nova coluna com ano e mês
df['Ano_Mes'] = df['Texto'].apply(extract_year_month)

# Exibir o DataFrame resultante
print(df)

                                                  Link  \
0    https://www.bcb.gov.br/en/publications/copommi...   
1    https://www.bcb.gov.br/en/publications/copommi...   
2    https://www.bcb.gov.br/en/publications/copommi...   
3    https://www.bcb.gov.br/en/publications/copommi...   
4    https://www.bcb.gov.br/en/publications/copommi...   
..                                                 ...   
178  https://www.bcb.gov.br/en/publications/copommi...   
179  https://www.bcb.gov.br/en/publications/copommi...   
180  https://www.bcb.gov.br/en/publications/copommi...   
181  https://www.bcb.gov.br/en/publications/copommi...   
182  https://www.bcb.gov.br/en/publications/copommi...   

                                     Texto  \
0       266th Meeting - November 5-6, 2024   
1    265th Meeting - September 17-18, 2024   
2         264th Meeting - July 30-31, 2024   
3         263rd Meeting - June 18-19, 2024   
4            262nd Meeting - May 7-8, 2024   
..                         

Usa o regex para encontrar essas expressoes, q sao expressoes que comecam a primeira secao (modelo de ata novo e antigo) e extrai o conteudo a partir dai. Alem disso cria novas variaveis com as secoes novas (A,B,C,D)

In [9]:
# Definir as expressões que você deseja encontrar
expressions = [
    r'(?i)(A\)\s*update (?:e conomic|economic) outlook (?:c opoms|copoms) (?:reference|baseline|)\s*scenario)(.*)',
    r'(?i)(Recent Economic Developments)(.*)',
    r'(?i)(Recent Evolution Inflation)(.*)',
]

# Função para extrair texto a partir de várias expressões
def extract_from_expressions(text):
    for pattern in expressions:
        match = re.search(pattern, text)
        if match:
            # Retorna o texto a partir da expressão encontrada até o final
            return text[match.start():]  # A partir do início da expressão até o final do texto
    return text  # Retorna todo o texto se nenhuma expressão for encontrada

df['Texto Processado'] = df['Texto Processado'].apply(extract_from_expressions)


# Definir os títulos das seções, permitindo variações com ou sem espaço
section_titles = [
    r'(?i)A\)\s*update (?:e conomic|economic) outlook (?:c opoms|copoms) (?:reference|baseline|)\s*scenario.*?(?=B\)\s*risks around|B\)\s*scenarios risk analysis|$)',
    r'(?i)(B\)\s*risks around|B\)\s*scenarios risk analysis).*?(?=C\)\s*discussion conduct monetary policy|$)',
    r'(?i)C\)\s*discussion conduct monetary policy.*?(?=D\)\s*monetary policy decision|$)',
    r'(?i)D\)\s*monetary policy decision.*?(?=Footnotes|$)',
    r'(?i)Footnotes.*'  # Captura tudo até o final do texto
]


# Criar variáveis para cada seção no DataFrame
for title in section_titles:
    # Usar str.extract para capturar as seções
    section_name = re.search(r'A|B|C|D|Footnotes', title).group(0)  # Captura A, B, C, D ou Footnotes
    df[section_name] = df['Texto Processado'].str.extract(f'({title})')[0]

print (df[['Texto Processado']])
# Exibir o DataFrame resultante
#print(df[['A', 'B', 'C', 'D', 'Footnotes']])

                                      Texto Processado
0    a) update economic outlook copoms scenario glo...
1    a) update economic outlook copoms scenario glo...
2    a) update economic outlook copoms scenario glo...
3    a) update economic outlook copoms scenario glo...
4    a) update economic outlook copoms scenario glo...
..                                                 ...
178  recent evolution inflation ipca increased incr...
179  recent evolution inflation inflation rates ref...
180  recent evolution inflation inflation continued...
181  recent evolution inflation measures inflation ...
182  recent evolution inflation measures inflation ...

[183 rows x 1 columns]


In [10]:
# Função para preprocessar o texto
def preprocess_text(text):
    if text:
        # Remove números
        text = re.sub(r'[0-9]+', '', text)

        # Remove pontuação e caracteres especiais, incluindo parênteses
        text = re.sub(r'[^\w\s]+', '', text)  # Remove toda pontuação, exceto letras e números

        # Substituir múltiplos espaços por um único espaço
        text = re.sub(r'\s+', ' ', text)

        # Retorna texto em minúsculas
        text = text.strip().lower()

        # Remove palavras com 1 letra
        text = ' '.join([word for word in text.split() if len(word) > 1])

        # Remove expressões específicas
        for expr in expressions_to_remove:
            text = text.replace(expr, '')

        # Remove stopwords
        text = ' '.join([word for word in text.split() if word not in stop_words])

        return text

    return ''

    # Criar a nova coluna 'Texto Processado'
df['Texto Processado2'] = df['Texto Processado'].apply(preprocess_text)

Agrupa pelo trimestre

In [11]:
#Cria a variavel trimestre
df['trimestre'] = df['Ano_Mes'].apply(lambda x: pd.to_datetime(x).to_period('Q'))
#print(df)

#Agrupando pelo trimestre
cols_to_join = [col for col in df.columns if col not in ['trimestre']]
agg_dict = {}
for col in cols_to_join:
    agg_dict[col] = lambda x: ' '.join(x.astype(str))

df_agrupado = df.groupby('trimestre').agg(agg_dict)
df_agrupado = df_agrupado.reset_index()

print(df_agrupado)

   trimestre                                               Link  \
0     2003Q2  https://www.bcb.gov.br/en/publications/copommi...   
1     2003Q3  https://www.bcb.gov.br/en/publications/copommi...   
2     2003Q4  https://www.bcb.gov.br/en/publications/copommi...   
3     2004Q1  https://www.bcb.gov.br/en/publications/copommi...   
4     2004Q2  https://www.bcb.gov.br/en/publications/copommi...   
..       ...                                                ...   
82    2023Q4  https://www.bcb.gov.br/en/publications/copommi...   
83    2024Q1  https://www.bcb.gov.br/en/publications/copommi...   
84    2024Q2  https://www.bcb.gov.br/en/publications/copommi...   
85    2024Q3  https://www.bcb.gov.br/en/publications/copommi...   
86    2024Q4  https://www.bcb.gov.br/en/publications/copommi...   

                                                Texto  \
0   85th Copom minutes - June 2003 84th Copom minu...   
1   88th Copom minutes - September 2003 87th Copom...   
2   91st Copom minutes -

In [12]:
#https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/9300-contas-nacionais-trimestrais.html?=&t=resultados

# Fazendo upload do arquivo XLS
uploaded = files.upload()

# Carregar as abas, iniciando da 5ª linha e sem considerar a 5ª linha como cabeçalho
df_taxa_trimestral = pd.read_excel('Tab_Compl_CNT.xls', sheet_name='Taxa Trimestral', engine='xlrd', skiprows=4, header=None)
df_trimestre_contra_trimestre = pd.read_excel('Tab_Compl_CNT.xls', sheet_name='Trimestre contra Trimestre Ant.', engine='xlrd', skiprows=4, header=None)

# Realizar o merge pela coluna A
df_trimestres_merged = pd.merge(df_taxa_trimestral, df_trimestre_contra_trimestre, on=0, how='outer')

# Selecionar as colunas Q de ambas as abas
df_trimestres_merged = df_trimestres_merged[[0, '17_x', '16_y']]

# Renomear as colunas 0, 17_x e 16_y para nomes mais descritivos
df_trimestres_merged = df_trimestres_merged.rename(columns={0: 'trimestre', '17_x': 'PIB_Taxa_Trimestral', '16_y': 'PIB_Trimestre_Contra_Trimestre'})

# Dicionário para converter números romanos para números árabes
romano_para_árabe = {
    'I': 1,
    'II': 2,
    'III': 3,
    'IV': 4
}

# Função para converter os trimestres romanos em formato 'YYYYQn'
def converter_trimestre(trimestre):
    if isinstance(trimestre, str):
        ano, trimestre_romano = trimestre.split('.')  # Divide o ano e o número romano
        trimestre_normal = romano_para_árabe.get(trimestre_romano, None)  # Converte o número romano para número árabe
        if trimestre_normal:
            return f"{ano}Q{trimestre_normal}"  # Formata como 'YYYYQn'
    return trimestre

# Aplicar a conversão à coluna 'trimestre'
df_trimestres_merged['trimestre'] = df_trimestres_merged['trimestre'].apply(converter_trimestre)

# Converter a coluna 'trimestre' para o tipo 'period[Q]'
df_trimestres_merged['trimestre'] = pd.to_datetime(df_trimestres_merged['trimestre']).dt.to_period('Q')

# Criar colunas com defasagens
df_trimestres_merged['PIB_Taxa_Trimestral_lag1'] = df_trimestres_merged['PIB_Taxa_Trimestral'].shift(1)
df_trimestres_merged['PIB_Taxa_Trimestral_lag2'] = df_trimestres_merged['PIB_Taxa_Trimestral'].shift(2)
df_trimestres_merged['PIB_Trimestre_Contra_Trimestre_lag1'] = df_trimestres_merged['PIB_Trimestre_Contra_Trimestre'].shift(1)
df_trimestres_merged['PIB_Trimestre_Contra_Trimestre_lag2'] = df_trimestres_merged['PIB_Trimestre_Contra_Trimestre'].shift(2)

print(df_trimestres_merged)

Saving Tab_Compl_CNT.xls to Tab_Compl_CNT.xls
    trimestre  PIB_Taxa_Trimestral  PIB_Trimestre_Contra_Trimestre  \
0      1996Q1            -0.579873                             NaN   
1      1996Q2             1.347193                        1.274862   
2      1996Q3             5.647213                        3.464082   
3      1996Q4             2.274484                       -0.740670   
4      1997Q1             3.400572                        1.009998   
..        ...                  ...                             ...   
109    2023Q2             3.468068                        0.711565   
110    2023Q3             1.959727                        0.129680   
111    2023Q4             2.053173                        0.185758   
112    2024Q1             2.456020                        1.041674   
113    2024Q2             3.335182                        1.447486   

     PIB_Taxa_Trimestral_lag1  PIB_Taxa_Trimestral_lag2  \
0                         NaN                       Na

<ipython-input-12-ee21e64f4acb>:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_trimestres_merged['trimestre'] = pd.to_datetime(df_trimestres_merged['trimestre']).dt.to_period('Q')


In [13]:
# Agora, você pode fazer o merge
df_final = pd.merge(df_agrupado, df_trimestres_merged[['trimestre', 'PIB_Taxa_Trimestral', 'PIB_Trimestre_Contra_Trimestre','PIB_Taxa_Trimestral_lag1','PIB_Taxa_Trimestral_lag2','PIB_Trimestre_Contra_Trimestre_lag1','PIB_Trimestre_Contra_Trimestre_lag2']], on='trimestre', how='outer')

#drop de 2003-03 para tras
# Filtra para manter apenas os registros a partir de 2003-03
df_final = df_final[df_final['trimestre'] >= '2003Q2']

# Verifique o resultado final
print(df_final.head())

   trimestre                                               Link  \
29    2003Q2  https://www.bcb.gov.br/en/publications/copommi...   
30    2003Q3  https://www.bcb.gov.br/en/publications/copommi...   
31    2003Q4  https://www.bcb.gov.br/en/publications/copommi...   
32    2004Q1  https://www.bcb.gov.br/en/publications/copommi...   
33    2004Q2  https://www.bcb.gov.br/en/publications/copommi...   

                                                Texto  \
29  85th Copom minutes - June 2003 84th Copom minu...   
30  88th Copom minutes - September 2003 87th Copom...   
31  91st Copom minutes - December 2003 90th Copom ...   
32  94th Copom minutes - March 2004 93rd Copom min...   
33  97th Copom minutes - June 2004 96th Copom minu...   

                                     Link de Download  \
29  https://www.bcb.gov.br/content/copom/copomminu...   
30  https://www.bcb.gov.br/content/copom/copomminu...   
31  https://www.bcb.gov.br/content/copom/copomminu...   
32  https://www.bcb.gov.br

Exportando para um arquivo xlsx ou csv (csv com problema de configuracao)

In [14]:
#exportando para um arquivo csv
#print(df_final.head())
#output_csv = 'dados_extraidos.csv'  # Nome do arquivo CSV
#df_final.to_csv(output_csv, index=False)

# Exportando para Excel
output_excel = 'dados_extraidos.xlsx'
df_final.to_excel(output_excel, index=False)

# Download no Google Colab
files.download(output_excel)

# Se você estiver usando Google Colab, faça o download do arquivo CSV
#try:
    #from google.colab import files
    #files.download(output_csv)
#except ImportError:
    #print("Não é um ambiente Colab. O arquivo está salvo como:", output_csv)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

--------------------------------------------------------------------------------

Dados pib antigo - Nao estou usando mais


In [ ]:
# https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/9300-contas-nacionais-trimestrais.html?=&t=series-historicas&utm_source=landing&utm_medium=explica&utm_campaign=pib#evolucao-taxa

# Fazendo upload do arquivo CSV
uploaded = files.upload()

# Carregar o arquivo CSV usando o delimitador correto
df_trimestres = pd.read_csv('pib_trimestral.csv', delimiter=';', header=1)  # O cabeçalho correto

# Excluir a primeira coluna se necessário (baseado no que você descreveu)
df_trimestres = df_trimestres.iloc[:1, 1:]  # Remove a primeira coluna


# Verifique o DataFrame após as alterações
print("DataFrame após a leitura:")
print(df_trimestres.head())

# Agora, precisamos reorganizar os dados
# Vamos usar melt para transformar os dados
df_trimestres_melted = df_trimestres.melt(var_name='Trimestre', value_name='Valor')

# Verifique o DataFrame derretido
print("DataFrame derretido:")
print(df_trimestres_melted.head())

# Função para gerar mapeamento de trimestres
def gerar_mapeamento_trimestres(inicio_ano, fim_ano):
    mapeamento = {}
    for ano in range(inicio_ano, fim_ano + 1):
        for trimestre in range(1, 5):
            mes = trimestre * 3  # 1: Março, 2: Junho, 3: Setembro, 4: Dezembro
            ano_mes = f"{ano}-{mes:02d}"
            mapeamento[f"{trimestre}º trimestre {ano}"] = ano_mes
    return mapeamento

# Gerar mapeamento de 1996 a 2024
mapeamento_trimestre = gerar_mapeamento_trimestres(1996, 2024)
df_trimestres_melted['Ano_Mes'] = df_trimestres_melted['Trimestre'].map(mapeamento_trimestre)

print("DataFrame com o mapeamento:")
print(df_trimestres_melted[['Trimestre', 'Ano_Mes']].head(20))  # Exibir as primeiras 20 linhas

# Verificar se há valores ausentes na coluna Ano_Mes
valores_ausentes = df_trimestres_melted['Ano_Mes'].isna().sum()
print(f"Número de valores ausentes em 'Ano_Mes': {valores_ausentes}")


# Fazer upload do segundo arquivo CSV
uploaded2 = files.upload()

# Carregar o segundo arquivo CSV
df_trimestres2 = pd.read_csv('pib_trimestral2.csv', delimiter=';', header=1)  # Ajuste o delimitador e o cabeçalho conforme necessário

# Excluir a primeira coluna no segundo DataFrame, se necessário
df_trimestres2 = df_trimestres2.iloc[:1, 1:]  # Remove a primeira coluna

# Transformar os dados do segundo DataFrame usando melt
df_trimestres2_melted = df_trimestres2.melt(var_name='Trimestre', value_name='Valor2')

# Gerar novamente o mapeamento de trimestres
df_trimestres2_melted['Ano_Mes'] = df_trimestres2_melted['Trimestre'].map(mapeamento_trimestre)



# Exibir trimestres e seus respectivos Ano_Mes
#for trimestre in df_trimestres_melted['Trimestre'].unique():
#    print(f"{trimestre}: {df_trimestres_melted[df_trimestres_melted['Trimestre'] == trimestre]['Ano_Mes'].values}")


# Verifique o resultado do mapeamento
#print("Trimestres únicos:")
#print(df_trimestres_melted['Trimestre'].unique())
#print(df_trimestres_melted[['Ano_Mes', 'Valor']])


# Agora, mescle os dois dataframes (supondo que o df original esteja carregado em df)
df_trimestres_prov = pd.merge(df_trimestres_melted[['Ano_Mes', 'Valor']], df_trimestres2_melted[['Ano_Mes', 'Valor2']], on='Ano_Mes', how='outer')
df_final = pd.merge(df, df_trimestres_prov[['Ano_Mes', 'Valor', 'Valor2']], on='Ano_Mes', how='outer')
#df_final = df_final.dropna(subset=['Texto Processado'])


#drop de 2003-03 para tras
# Filtra para manter apenas os registros a partir de 2003-03
df_final = df_final[df_final['Ano_Mes'] >= '2003-04']

df_final['trimestre'] = df_final['Ano_Mes'].apply(lambda x: pd.to_datetime(x).to_period('Q'))

# Verifique o resultado final
print(df_final.head())

Saving pib_trimestral.csv to pib_trimestral.csv
DataFrame após a leitura:
   1º trimestre 1996  2º trimestre 1996  3º trimestre 1996  4º trimestre 1996  \
0                2.5                2.1                2.3                2.2   

   1º trimestre 1997  2º trimestre 1997  3º trimestre 1997  4º trimestre 1997  \
0                3.2                4.0                3.0                3.4   

   1º trimestre 1998  2º trimestre 1998  ...  1º trimestre 2022  \
0                2.8                2.0  ...                4.7   

   2º trimestre 2022  3º trimestre 2022  4º trimestre 2022  1º trimestre 2023  \
0                2.7                2.7                3.0                3.7   

   2º trimestre 2023  3º trimestre 2023  4º trimestre 2023  1º trimestre 2024  \
0                3.7                3.1                2.9                2.5   

   2º trimestre 2024  
0                2.5  

[1 rows x 114 columns]
DataFrame derretido:
           Trimestre  Valor
0  1º trimestre 1996

Saving pib_trimestral2.csv to pib_trimestral2.csv
                                                 Link  \
29  https://www.bcb.gov.br/en/publications/copommi...   
30  https://www.bcb.gov.br/en/publications/copommi...   
31  https://www.bcb.gov.br/en/publications/copommi...   
32  https://www.bcb.gov.br/en/publications/copommi...   
33  https://www.bcb.gov.br/en/publications/copommi...   

                                  Texto  \
29        84th Copom minutes - May 2003   
30       85th Copom minutes - June 2003   
31       86th Copom minutes - July 2003   
32     87th Copom minutes - August 2003   
33  88th Copom minutes - September 2003   

                                     Link de Download  \
29  https://www.bcb.gov.br/content/copom/copomminu...   
30  https://www.bcb.gov.br/content/copom/copomminu...   
31  https://www.bcb.gov.br/content/copom/copomminu...   
32  https://www.bcb.gov.br/content/copom/copomminu...   
33  https://www.bcb.gov.br/content/copom/copomminu...   

     